<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Skriptsprachen
### Sommersemester 2021
Prof. Dr. Heiner Giefers

# Covid-19 Daten visualisieren

In dieser kleinen Aufgabe geht es darum, die weltweiten Covid-19 Zahlen zu visualisieren.
Wir wollen die die Daten von einer URL herunterladen und als *Pandas DataFrame* anlegen.
Damit können wir die Daten vorverarbeiten und schließlich auch plotten.

In [ ]:
import numpy as np
import csv
import requests
import io
import pandas as pd
import plotly.express as px
import geojson

In [ ]:
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [ ]:
print(f"Die Anzahl aller Einträge ist {df.size}")   

Um sich einen ersten Eindruck von der Tabelle zu machen, kann man eine Reihe von Pandas-Methoden aufrufen:
- `df.head(k)` zeigt die ersten `k` Einträge der Tabelle. Sie werden sehen, dass die Daten nach Ländern sortiert sind
- `df.info()` zeigt Informationen zu den Spalten der Tabelle
- `df.describe()` Gibt einige statistische Kennzahlen zu den Daten aus

In [ ]:
df.head(10)

Wenn Sie die Daten nach einer anderen Spalte sortieren wollen, geht das mit der `sort_by_values` Methode:

In [ ]:
df_date = df.sort_values(by='date')
df_date

Wir wollen nun das in Deutschland häufig verwendete Maß der *7-Tage Inzidenz* darstellen. Dieses ist in der Tabelle aber nicht direkt enthalten.
Wir können es allerdings aus den neuen Fällen pro Tag berechnen. Um eine Normalisierung gemäß der Einwohnerzahlen zu erreichen, verwendenden wir die Spalte `new_cases_per_million`.
Diese kann allerdings fehlende Werte enthalten, z.B. weil für einige Länder an bestimmten Tagen keine Daten vorlagen.
Um diese fehlenden Werte zu *schätzen*, interpolieren wir. D.h. wir nehmen an, bei einer *Lücke* würden sie Werte linear fortlaufen. Also bei der folge `1, 2, 3, NaN, 7, 8, 9` würde das `NaN` durch `5` ersetzt.

Nun gibt es noch ein weiteres Problem: Wir haben die Tabelle nach Daten sortiert, alle Länder stehen also vermischt in der Tabelle.
Beim Aufsummieren der *7-Tage-Inzidenz* sollen aber natürlich nur Daten innerhalb eines Landes betrachtet werden.
Um dies zu erreichen, können wir die `groupby`-Methode verwenden.
Als Parameter erfhält `groupby` eine Funktion, die auf die gruppierten Daten angewendet wird.

**Aufgabe:** Implementieren Sie die Funktion `berechne_inzidenz(x)`, die dem DataFrame `x` eine Spalte `Inzidenz` hinzufügt. Dazu soll zuerst die Spalte `new_cases_per_million` mit der Funktion `interpolate()` interpoliert werden. Anschließend soll die 7-Tage Inzidenz ausgerechnet werden. Sie können dazu die Methode `rolling(k)` verwenden, die ein gleitendes *Fenster* über `k`-Werte der Spalte liefert.

In [ ]:
# Falsche Werte aussortieren
indexEntries = df_date[df_date['new_cases_per_million'] < 0 ].index
df_cleaned = df_date.drop(indexEntries)

def berechne_inzidenz(x):
    # YOUR CODE HERE
    raise NotImplementedError()
    return x
    
df_cleaned = df_cleaned.groupby('iso_code').apply(berechne_inzidenz)

Nun können wir die Inzidenz-Werte anzeigen.
Dafür eignet sich gut eine Darstellung als Weltkarte, die wir z.B. mit der *Plotly* Methode `choropleth` erzeugen können.

In [ ]:
fig = px.choropleth(df_cleaned, locations="iso_code",
                    color="Inzidenz",
                    #scope='europe',
                    range_color = [0,200],
                    hover_name="location",
                    animation_frame="date",
                    title = "Corvid: weltweite 7-Tages Inzidenz",
                    color_continuous_scale=px.colors.sequential.Jet)
 
 
fig["layout"].pop("updatemenus")
fig.show()

Um bestimmte Zeilen eines DataFrames herauszufiltern, kann man bei der Auswahl der Spalten Bedingungen angeben.
So können wir z.B. die Werte aus Deutschland aus der Tabelle herausfiltern:

In [ ]:
df_de = df[df['iso_code']=='DEU']

print(f"Die Anzahl aller Einträge aus Deutschland ist {df_de.size}")
df_de.head()

**Aufgabe:** Plotten Sie die Inzidenz-Werte für Deutschland (`DEU`), Großbritannien (`GBR`) und USA (`USA`) in einen gemeinsamen Graphen.
Verwenden sie dau die *Matplotlib*-Methode `plot()`.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

fig, axes = plt.subplots(1,1,figsize=(16,8))

df_cleaned['dedate'] =  pd.to_datetime(df_cleaned.date).dt.strftime('%d.%m.%Y')
# YOUR CODE HERE
raise NotImplementedError()
axes.xaxis.set_major_locator(MaxNLocator(15))
plt.xticks(rotation = 45) 
plt.legend()
plt.show()

## Datenquelle

Die verwendeten Daten stammen von [_Our World in Data_](https://ourworldindata.org/) und wurden dem Git-Repository [https://github.com/owid/covid-19-data](https://github.com/owid/covid-19-data) entnommen

Details zum Datensatz findet man in der folgenden Publikation:

> Hasell, J., Mathieu, E., Beltekian, D. _et al._ A cross-country database of COVID-19 testing. _Sci Data_ **7**, 345 (2020). [https://doi.org/10.1038/s41597-020-00688-8](https://doi.org/10.1038/s41597-020-00688-8)

